# Fully connected NN with tensorflow

First we import the necessary modules

In [1]:
import tensorflow as tf
import numpy as np
import math

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from data_handler_CMAPS import CMAPSDataHandler

from numpy.random import seed
from tensorflow import set_random_seed

/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [7]:
def print_model_weights(sess):
    for v in tf.trainable_variables():
        
        layer = sess.run(v)
        
        print(v.name)
        print(layer.shape)
        print(layer)

# Function to train the neural network

In [10]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

def multilayer_perceptron(X):
    
    l2_regularizer = 0.0
    
    X = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                    kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=0), 
                    kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_regularizer))
    
    X = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                    kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=0), 
                    kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_regularizer))
    
    X = tf.layers.dense(X, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                       kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_regularizer))
    
    return X 

def multilayer_perceptron2(X):
    
    l2_regularizer = 0.0
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=0), name="fc1")
    
    y = tf.layers.dense(A1, 1, activation=None, 
                        kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=0), name="out")
    
    return y


def get_minibatches(X_full, y_full, batch_size):
    
    full_size = X_full.shape[0]
    total_batches = math.floor(full_size/batch_size)
    remainder = full_size - total_batches*batch_size
    
    X_batches = []
    y_batches = []
    
    for i in range(total_batches):
        X_batches.append(X_full[i*batch_size:(i+1)*batch_size])
        y_batches.append(y_full[i*batch_size:(i+1)*batch_size])
        
    if remainder != 0:
        X_batches.append(X_full[total_batches*batch_size:])
        y_batches.append(y_full[total_batches*batch_size:])
        total_batches = total_batches+1
        
    return X_batches, y_batches, total_batches
        

def train_ann(X_train, y_train, epochs, batch_size, display_step = 1, lamba2 = 0.0):
    
    seed(0)
    set_random_seed(0)
    
    input_shape = X_train.shape[1]
    output_shape = 1
    
    X, y = create_placeholders(input_shape, output_shape)
    
    predictions = multilayer_perceptron2(X)
    cost = tf.losses.mean_squared_error(y, predictions) 
    reg_cost = tf.losses.get_regularization_loss()
    total_cost = cost + reg_cost
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(total_cost)
    
    with tf.Session() as sess:
        
        #To reset all variables
        sess.run(tf.global_variables_initializer())
        avg_cost = 0.0
        
        print_model_weights(sess)
        
        for epoch in range(epochs):
            
            cost_tot = 0.0
            cost_reg_tot = 0.0
            #total_batch = int(X_train.shape[0]/batch_size)
            #X_batches = np.array_split(X_train, total_batch)
            #y_batches = np.array_split(y_train, total_batch)
            
            X_batches, y_batches, total_batch = get_minibatches(X_train, y_train, batch_size)
            
            #Train with the minibatches
            for i in range(total_batch):
                batch_x, batch_y = X_batches[i], y_batches[i]
                
                #print(batch_x.shape)
                
                _, c_reg, c, reg_cost_val = sess.run([optimizer, total_cost, cost, reg_cost], feed_dict={X:batch_x, y:batch_y})
                cost_tot += c
                cost_reg_tot += c_reg
                
                #print(reg_cost_val)
                
            avg_cost = cost_tot/total_batch
            avg_cost_reg = cost_reg_tot/total_batch
                
            if epoch%display_step == 0:
                print("Epoch:", '%04d' % (epoch+1), "cost_reg=", "{:.9f}".format(avg_cost_reg), "cost=", "{:.9f}".format(avg_cost))
                    
        print("Training complete!")
        print("Epoch:Final", "cost_reg=", "{:.9f}".format(avg_cost_reg), "cost=", "{:.9f}".format(avg_cost)) 

# Create the Data Handler

In [8]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 128

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul=max_rul,
                                  sequence_length=window_size, sequence_stride=window_stride)

dHandler_cmaps.load_data(unroll=True, verbose=1, cross_validation_ratio=0)

#Rescale the data
dHandler_cmaps.X_train = min_max_scaler.fit_transform(dHandler_cmaps.X_train)
dHandler_cmaps.X_test = min_max_scaler.transform(dHandler_cmaps.X_test)

dHandler_cmaps.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ..., -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ..., -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ..., -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ..., -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ..., -0.45454545  0.05511811
   0.17880983]]
[[ 128.]
 [ 128.]
 [ 128.]
 [ 128.]
 [ 128.]]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ..., -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ..., -0.63636364  0.05511811
   0.04416986]
 [ 0.13402062 -0.08644501  0.038854   ...,  0.09090909 

# Train the ANN

In [9]:
window_size = 30
window_stride = 1
max_rul = 128
num_features = len(selected_features)
shape = num_features*window_size

#Train the model
X = dHandler_cmaps.X_train
y = dHandler_cmaps.y_train

epochs = 20
batch_size = 512

tf.reset_default_graph()

model = train_ann(X, y, epochs=epochs, batch_size=batch_size, display_step = 1)

fc1/kernel:0
(420, 20)
[[ 0.00952573 -0.13807291  0.02388828 ...,  0.0425049   0.11824349
   0.00237017]
 [-0.06897566  0.10282086 -0.01936359 ...,  0.04727668  0.04241179
  -0.1119597 ]
 [ 0.05336403  0.03035624  0.02375947 ...,  0.05647624  0.06885112
   0.11151468]
 ..., 
 [-0.07528567  0.01343323  0.01169548 ...,  0.07860185 -0.05693797
   0.0197363 ]
 [ 0.00382707 -0.00654522  0.04621871 ..., -0.08136505 -0.02641646
   0.10063449]
 [-0.00072989  0.12355479 -0.05378776 ...,  0.00576838 -0.0329321
   0.03504931]]
fc1/bias:0
(20,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
out/kernel:0
(20, 1)
[[ 0.04360284]
 [-0.63201177]
 [ 0.10934566]
 [-0.46428487]
 [ 0.19505545]
 [-0.33840579]
 [-0.20801905]
 [-0.468016  ]
 [-0.36447951]
 [ 0.17511755]
 [-0.31545013]
 [-0.41481665]
 [-0.36223534]
 [ 0.23793462]
 [ 0.3529304 ]
 [-0.43133992]
 [ 0.65901202]
 [ 0.19456096]
 [ 0.54124498]
 [ 0.01084916]]
out/bias:0
(1,)
[ 0.]
Epoch: 0001 cost_reg= 7819.0792271